In [ ]:
import pandas as pd
import nltk
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')
from nltk.corpus import stopwords
from collections import  Counter
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import re

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
NY_reviews = pd.read_csv('/content/drive/MyDrive/DS4A Team4/New_York_reviews_cleaned.csv', index_col=0)
NY_reviews.head()

,restaurant_name,rating_review,sample,title_review,review_preview,review_full,date,author_id
1,Lido,5,Positive,a regular treat,my wife and i have been eating dinner frequent...,my wife and i have been eating dinner frequent...,2020-10-08,UID_0
2,Lido,4,Positive,good neighborhood spot!,came with family for labor day weekend brunch ...,came with family for labor day weekend brunch ...,2020-09-08,UID_1
3,Lido,1,Negative,disappointing,food was mediocre at best. the lamb chops are...,food was mediocre at best. the lamb chops are ...,2020-02-17,UID_2
4,Lido,5,Positive,what a find in harlem,my co-workers were volunteering at a foodbank ...,my co-workers were volunteering at a foodbank ...,2019-11-25,UID_3
5,Lido,5,Positive,lunch,lido is an intimate boutique style restaurant....,lido is an intimate boutique style restaurant....,2019-11-23,UID_4


In [ ]:
k = 2
# number of latest reviews / user in Test
### Train / Test split:
## Train: part 1 - users with less than k+1 reviews
train_1 = NY_reviews.groupby("author_id").filter(lambda x: len(x) <= k)

## Train: part 2 - users with at least k reviews: all reviews except the latest k reviews
users_with_more_than_k_reviews = NY_reviews.groupby("author_id").filter(lambda x: len(x) > k)
# Sort the DataFrame by 'author_id' and 'date' in descending order
users_with_more_than_k_reviews_sorted = users_with_more_than_k_reviews.sort_values(by=['author_id', 'date'], ascending=[True, False])
# Get indexes of latest k reviews
def get_latest_k_reviews(group):
    return group.index[:k]
latest_reviews_indexes = users_with_more_than_k_reviews_sorted.groupby('author_id', group_keys=False).apply(get_latest_k_reviews).values
latest_reviews_indexes_list = [item for sublist in latest_reviews_indexes for item in sublist]

train_2 = users_with_more_than_k_reviews_sorted[~users_with_more_than_k_reviews_sorted.index.isin(latest_reviews_indexes_list)]
train = pd.concat([train_1, train_2])
test = users_with_more_than_k_reviews_sorted.loc[latest_reviews_indexes_list]

In [ ]:
grouped = train[['restaurant_name','review_full']].groupby('restaurant_name', as_index=False).agg(' '. join)

In [ ]:
query = train[['author_id','review_full']].groupby('author_id', as_index=False).agg(' '. join)

In [ ]:
df = grouped
df.columns = ['restaurant_name','sentence']
query.columns = ['author_id','sentence']

## Bert

In [ ]:
pip install -U sentence-transformers

In [ ]:
from sentence_transformers import SentenceTransformer, util

model = SentenceTransformer('all-mpnet-base-v2')
corpus_embeddings = model.encode(df.sentence.values, convert_to_tensor=True)
query_embedding = model.encode(query.sentence.values, convert_to_tensor=True)

In [ ]:
corpus_embeddings.size()

In [ ]:
query_embedding.size()

In [ ]:
restaurant_reviews_embedding_NM = pd.DataFrame(corpus_embeddings.cpu())

In [ ]:
user_embedding_NM = pd.DataFrame(query_embedding.cpu())

In [ ]:
restaurant_reviews_embedding_NM.to_csv('/content/drive/MyDrive/restaurant_reviews_embedding_NM.csv',  sep=',')

In [ ]:
user_embedding_NM.to_csv('/content/drive/MyDrive/user_embedding_NM.csv',  sep=',')

In [ ]:
cos_scores = util.pytorch_cos_sim(query_embedding, corpus_embeddings)

In [ ]:
cos_scores.size()

In [ ]:
cosine_similarity_matrix = pd.DataFrame(cos_scores.cpu())

In [ ]:
cosine_similarity_matrix

In [ ]:
cosine_similarity_matrix.to_csv('/content/drive/MyDrive/cosine_similarity_matrix_NM.csv',  sep=',')

## Evaluation

In [ ]:
cosine_similarity_matrix = pd.read_csv('/content/drive/MyDrive/cosine_similarity_matrix_NM.csv',  index_col=0)

In [ ]:
cosine_similarity_matrix

,0,1,2,3,4,5,6,7,8,9,...,1718,1719,1720,1721,1722,1723,1724,1725,1726,1727
0,0.585372,0.464674,0.407267,0.612275,0.496679,0.419017,0.454926,0.359988,0.482329,0.422314,...,0.496331,0.509202,0.406944,0.566599,0.317521,0.622984,0.509466,0.397255,0.557202,0.501754
1,0.788216,0.582752,0.569701,0.638326,0.573150,0.553287,0.654353,0.487378,0.622191,0.680610,...,0.621403,0.686462,0.661369,0.732134,0.557951,0.705632,0.522129,0.634892,0.640007,0.509085
2,0.682950,0.661378,0.620428,0.636300,0.620176,0.566870,0.659030,0.583974,0.631063,0.621951,...,0.651837,0.663036,0.630787,0.670679,0.540883,0.659300,0.604600,0.687055,0.688230,0.545049
3,0.491467,0.499353,0.635585,0.580944,0.472954,0.246306,0.255378,0.241083,0.462041,0.485515,...,0.429629,0.590772,0.514352,0.469025,0.417435,0.524041,0.503587,0.318094,0.582949,0.333906
4,0.672770,0.587805,0.444975,0.642922,0.534393,0.412919,0.361617,0.395926,0.441660,0.663220,...,0.600814,0.573462,0.476658,0.639937,0.402938,0.675341,0.428638,0.409865,0.577793,0.466285
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
249896,0.446314,0.454778,0.393496,0.490917,0.456858,0.333795,0.507740,0.298657,0.475949,0.354527,...,0.396050,0.488790,0.463035,0.461627,0.382518,0.530042,0.344246,0.564893,0.431175,0.255842
249897,0.707976,0.532251,0.408648,0.595110,0.651102,0.594517,0.541912,0.512112,0.543850,0.531434,...,0.523032,0.634074,0.527040,0.677253,0.348129,0.716448,0.392094,0.456590,0.573674,0.553691
249898,0.634906,0.557396,0.574470,0.582769,0.520889,0.554193,0.477965,0.589106,0.586485,0.537327,...,0.641179,0.502630,0.615771,0.587872,0.460112,0.531793,0.377928,0.492197,0.625659,0.578864
249899,0.607844,0.467120,0.435449,0.520117,0.475195,0.290789,0.433669,0.384844,0.505713,0.432993,...,0.506849,0.524741,0.490065,0.496618,0.405064,0.520746,0.402910,0.504746,0.439101,0.391177


In [ ]:
test.head()

,restaurant_name,rating_review,sample,title_review,review_preview,review_full,date,author_id
231221,Don_Giovanni,5,Positive,locals' italian... great theatre district choice,"enjoy the delicious, well-priced food in this ...","enjoy the delicious, well-priced food in this ...",2020-02-26,UID_10
11,Havana_Central_Restaurant_Rum_Bar,5,Positive,a rollicking place to go for great food,a rollicking place to celebrate your birthday ...,a rollicking place to celebrate your birthday ...,2019-11-10,UID_10
144411,ABA_Turkish_Restaurant,4,Positive,"good food, good atmosphere",classic turkish dishes quite well done. the se...,classic turkish dishes quite well done. the se...,2014-10-28,UID_100002
267713,Shanghai_Asian_Cuisine,5,Positive,"great food, great value, tight quarters",we almost walked in to the fancier place next ...,we almost walked in to the fancier place next ...,2014-10-28,UID_100002
436678,Red_Lobster,3,Negative,take away order,take away orderi order the grilled salmon ...,take away orderi order the grilled salmon ...,2017-08-26,UID_100005


In [ ]:
### Check hit rate
count = 0
total_count = 0
for i in range(int(test.shape[0]/2)):
  user = test.iloc[i*2].author_id
  user_index = query.index[query['author_id']==user].tolist()[0]
  top6 = (cosine_similarity_matrix.iloc[user_index,:].sort_values(ascending = False))[:30]
  top6_rest = grouped.iloc[top6.index].restaurant_name.tolist()
  rest_1 = test.iloc[i*2].restaurant_name
  rating_1 = test.iloc[i*2].rating_review
  rest_2 = test.iloc[i*2+1].restaurant_name
  rating_2 = test.iloc[i*2+1].rating_review
  total_count += 2
  if rest_1 in top6_rest:
    count = count - 3 + rating_1
  if rest_2 in top6_rest:
    count = count - 3 + rating_2
print('count')
print(count)
print('total')
print(total_count)

postive
2232
total
44451


In [ ]:
positive_count = 0
negative_count = 0
same_rating_count = 0
total_count = 0
for i in range(int(test.shape[0]/2)):
  user = test.iloc[i*2].author_id
  user_index = query.index[query['author_id']==user].tolist()[0]
  rest_1 = test.iloc[i*2].restaurant_name
  rest_1_index = grouped.index[grouped['restaurant_name']==rest_1].tolist()
  if len(rest_1_index)!=0:
    rest_1_index = rest_1_index[0]
  else:
    continue
  rating_1 = test.iloc[i*2].rating_review
  rest_2 = test.iloc[i*2+1].restaurant_name
  rest_2_index = grouped.index[grouped['restaurant_name']==rest_2].tolist()
  if len(rest_2_index)!=0:
    rest_2_index = rest_2_index[0]
  else:
    continue
  total_count += 1
  rating_2 = test.iloc[i*2+1].rating_review
  model_result_1 = cosine_similarity_matrix.iloc[user_index,rest_1_index]
  model_result_2 = cosine_similarity_matrix.iloc[user_index,rest_2_index]
  if rating_1 == rating_2:
    same_rating_count += 1
    continue
  if (rating_1 > rating_2) and (model_result_1 >= model_result_2):
    positive_count += 1
  elif (rating_1 < rating_2) and (model_result_1 <= model_result_2):
    positive_count += 1
  elif (rating_1 > rating_2) and (model_result_1 < model_result_2):
    negative_count += 1
  elif (rating_1 < rating_2) and (model_result_1 > model_result_2):
    negative_count += 1
print('postive')
print(positive_count)
print('negative')
print(negative_count)
print('same ratings')
print(same_rating_count)
print('total')
print(total_count)

postive
12188
negative
12190
same ratings
19996
total
44374
